In [182]:
import math
import os
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from pandas import json_normalize
import yaml
from yaml import CSafeLoader as Loader

sns.set_context("paper", font_scale=3)

# LOG_DIR = os.path.expanduser("~/jean-zay/emergent_communication/lightning_logs_final/")
# LOG_DIR = os.path.expanduser("~/PhD/emergent_communication/emergent_communication/lightning_logs/")
LOG_DIR = os.path.expanduser("~/PhD/emergent_communication/emergent_communication/lightning_logs_cluster/")


REFERENCE_METRIC = "val_acc"

In [183]:
import torch

# Remove superfluous NaN cells
def compress(values):
    for val in values:
        if val is not None and not np.isnan(val):
            return val
        
results_df = []
for run_dir in tqdm(os.listdir(LOG_DIR)):
    run_path = os.path.join(LOG_DIR, run_dir, "checkpoints/")

    results = glob.glob(run_path+f"*{REFERENCE_METRIC}=*.pickle")
    if len(results) < 1:
        run_path = os.path.join(LOG_DIR, run_dir)
        results = glob.glob(run_path+f"/*.pickle")
        if len(results) >= 1:
            print(f"considering pre-final results for run {run_dir[8:]}")
            if run_dir[8:] == "31521":
                print("gre")

    for result in results:
        try:
            print(result)
            data = pd.read_pickle(result)
        except Exception as e:
            print(result)
            print(e)
            continue
            
        if isinstance(data, list):
            df = pd.DataFrame.from_records(data)
            df = df.groupby(["dir_name", "epoch"]).aggregate(compress)
        else:
            if "receiver_aux_loss" in data.keys():
                data["receiver_auxiliary_loss"] = data["receiver_aux_loss"]
                del data["receiver_aux_loss"]
            df = pd.DataFrame.from_records([{k: v.item() if torch.is_tensor(v) else v for k, v in data.items()}])
        
        df["dir_name"] = run_dir
        if "epoch" in result:
            df["epoch"] = int(result.split("epoch=")[1].split("-")[0])
        results_df.append(df)


results_df = pd.concat(results_df, ignore_index=True)

results_df.reset_index(inplace=True)
results_df

  0%|          | 0/297 [00:00<?, ?it/s]

considering pre-final results for run 42294
/home/mitja/PhD/emergent_communication/emergent_communication/lightning_logs_cluster/version_42294/results.pickle
/home/mitja/PhD/emergent_communication/emergent_communication/lightning_logs_cluster/version_40064/checkpoints/epoch=299-val_acc=0.90_results.pickle


  1%|          | 2/297 [00:00<00:32,  9.12it/s]

considering pre-final results for run 42214
/home/mitja/PhD/emergent_communication/emergent_communication/lightning_logs_cluster/version_42214/results.pickle
considering pre-final results for run 47547
/home/mitja/PhD/emergent_communication/emergent_communication/lightning_logs_cluster/version_47547/results.pickle
/home/mitja/PhD/emergent_communication/emergent_communication/lightning_logs_cluster/version_37437/checkpoints/epoch=179-val_acc=0.79_results.pickle


  2%|▏         | 5/297 [00:00<00:28, 10.12it/s]

...

 99%|█████████▊| 293/297 [00:10<00:00, 35.48it/s]

/home/mitja/PhD/emergent_communication/emergent_communication/lightning_logs_cluster/version_56030/checkpoints/epoch=2019-val_acc=0.73_results.pickle
/home/mitja/PhD/emergent_communication/emergent_communication/lightning_logs_cluster/version_41783/checkpoints/epoch=2099-val_acc=0.65_results.pickle
/home/mitja/PhD/emergent_communication/emergent_communication/lightning_logs_cluster/version_41776/checkpoints/epoch=3739-val_acc=0.62_results.pickle
/home/mitja/PhD/emergent_communication/emergent_communication/lightning_logs_cluster/version_22602/checkpoints/epoch=2259-val_acc=0.73_results.pickle
/home/mitja/PhD/emergent_communication/emergent_communication/lightning_logs_cluster/version_46210/checkpoints/epoch=3019-val_acc=0.92_results.pickle
/home/mitja/PhD/emergent_communication/emergent_communication/lightning_logs_cluster/version_56032/checkpoints/epoch=3339-val_acc=0.76_results.pickle
/home/mitja/PhD/emergent_communication/emergent_communication/lightning_logs_cluster/version_36561/c

100%|██████████| 297/297 [00:10<00:00, 27.83it/s]


/home/mitja/PhD/emergent_communication/emergent_communication/lightning_logs_cluster/version_41792/checkpoints/epoch=2359-val_acc=0.45_results.pickle


,index,receiver_loss,entropy_loss,sender_policy_loss,train_acc,receiver_loss/dataloader_idx_0,entropy_loss/dataloader_idx_0,sender_policy_loss/dataloader_idx_0,receiver_loss/dataloader_idx_1,entropy_loss/dataloader_idx_1,...,epoch,topsim_sender_receiver,posdis_sender_receiver,bosdis_sender_receiver,receiver_auxiliary_loss/dataloader_idx_0,receiver_aux_acc/dataloader_idx_0,receiver_auxiliary_loss/dataloader_idx_1,receiver_aux_acc/dataloader_idx_1,receiver_auxiliary_loss,receiver_aux_acc
0,0,0.365009,0.001697,-0.017670,0.783,0.385042,0.001370,0.014022,0.346393,0.001744,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,0.099775,0.000057,0.064865,NaN,NaN,...,299.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.058852,0.000051,...,299.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2.873114,0.000536,-0.007257,0.924,1.743315,0.000573,0.007537,1.568772,0.000796,...,NaN,0.268158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.514847,0.000570,-0.098860,0.697,0.496672,0.000474,-0.011489,0.523425,0.000460,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,542,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.627530,0.000640,...,59.0,0.113536,0.129548,0.056708,NaN,NaN,NaN,NaN,NaN,NaN
543,543,NaN,NaN,NaN,NaN,1.146802,0.000162,0.040328,NaN,NaN,...,1319.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
544,544,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.951738,0.000160,...,1319.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
545,545,NaN,NaN,NaN,NaN,1.471606,0.000291,0.076301,NaN,NaN,...,2359.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
hp = []
for run_dir in tqdm(os.listdir(LOG_DIR)):
    file_path = os.path.join(LOG_DIR, run_dir, "hparams.yaml")
    file = yaml.load(open(file_path), Loader=Loader) #safe_load(, Loader=Loader)
    df = json_normalize(file)
    df["dir_name"] = run_dir
    hp.append(df)

hp = pd.concat(hp, ignore_index=True)
hp

  0%|          | 0/297 [00:00<?, ?it/s]

In [ ]:
def fix_duplicate_value(val, allow_offset=None):
    if isinstance(val, list):
        for el in val:
            if allow_offset is None:
                assert (el == val[0]) or (el == "None") or (val[0] == "None")
            else:
                assert (np.abs(el - val[0]) < allow_offset) or (el == "None") or (val[0] == "None")
        return val[0]
    else:
        return val

hp = hp.applymap(fix_duplicate_value)

In [ ]:
indices_best_steps = results_df.groupby("dir_name")[REFERENCE_METRIC].idxmax()

df = results_df.loc[list(indices_best_steps)].copy()
df

In [ ]:
df.set_index("dir_name", inplace=True, drop=False)
if not hp.index.name == "dir_name":
    hp.set_index("dir_name", inplace=True, verify_integrity=True)
df = df.join(hp, how="left")

In [ ]:
assert (df.sender_entropy_coeff == df.receiver_entropy_coeff).all()
assert (df.num_senders == df.num_receivers).all()
assert (df.sender_layer_norm == df.receiver_layer_norm).all()

df["entropy_coeff"] = df["sender_entropy_coeff"]
df["num_agents"] = df["num_senders"]
df["layer_norm"] = df["sender_layer_norm"]

df["attr_val"] = df["num_attributes"].map(int).map(str) + "_" + df["num_values"].map(int).map(str)

data = df

In [ ]:
def calc_capacity(row):
    return math.pow(row.num_values, row.num_attributes)

data["capacity"] = data.apply(calc_capacity, axis=1)
data.sort_values("capacity", inplace=True)

In [ ]:
data["test_acc"] = data["val_acc"]
data["test_acc_no_noise"] = data["val_acc_no_noise"]

In [ ]:
data["condition"] = data["noise"].map(lambda x: f"noise_{x}" if x > 0 else "baseline") + data["feedback"].map(lambda x: "_feedback" if x else "") + data["receiver_aux_loss"].map(lambda x: "_aux_loss" if x else "")

target_data = data.copy()

target_data = target_data[target_data.receiver_aux_loss == False]

NUM_AGENTS = 1
target_data = target_data[target_data.num_agents == NUM_AGENTS]

MAX_LEN = 10
target_data = target_data[target_data.max_len == MAX_LEN]

NOISE = 0.5
target_data = target_data[target_data.noise == NOISE]

VOCAB_SIZE = 2
target_data = target_data[target_data.vocab_size == VOCAB_SIZE]

VOCAB_SIZE_FEEDBACK = 2
target_data = target_data[(target_data.vocab_size_feedback == VOCAB_SIZE_FEEDBACK) | (target_data.feedback == False)]

LAYER_NORM = 1
target_data = target_data[target_data.layer_norm == LAYER_NORM]

DISCRIMINATION_NUM_OBJECTS = 2
target_data = target_data[target_data.discrimination_num_objects == DISCRIMINATION_NUM_OBJECTS]

# hue_order = None
# hue_order = ["baseline", "noise_0.1"]
hue_order = ["noise_0.5", "noise_0.5_feedback", "noise_0.5_feedback_aux_loss"]

# order = ["2_10", "3_5", "2_16", "4_4", "4_5", "3_10", "2_100"]
# order = ["2_100", "4_100", "10_100"]
order = ["2_10", "4_4", "3_10", "2_100", "2_1000", "10_1000"]

target_hparam = "attr_val"

num_runs_data = target_data.groupby(["attr_val", "condition"]).size().reset_index()


metrics = ["test_acc", "test_acc_no_noise", "topsim"]
# metrics = ["test_acc", "topsim", "topsim_sender_receiver", "posdis", "bosdis", "test_acc_no_noise", "train_acc_no_noise"]
_, axes = plt.subplots(1, len(metrics), figsize=(35, 12))

for value in order:
    for fb in [False, True]:
        d = target_data[(target_data[target_hparam] == value) & (target_data["feedback"] == fb)]
        if len(d) != 3:
            print(f"Unexpected number of datapoints: {len(d)}")
            print(value)
            print(fb)
            print(d)

for i, metric in enumerate(metrics):
    axis = axes[i]
    # g = sns.barplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue="feedback", dodge=True)
    g = sns.pointplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue="feedback", errorbar="se", linestyles="", dodge=0.1)

    if not i == 0:
        g.legend().remove()
    else:
        sns.move_legend(axis, "lower left")
    axis.set_title(metric)
    axis.set_ylabel("")
    # handles, labels = ax.get_legend_handles_labels()
    # num_conditions = int(len(handles)/2) if not hue_order else len(hue_order)

plt.tight_layout()
plt.savefig("plots/effect_of_attr_val.pdf", dpi=300)


In [ ]:
target_data = data.copy()

target_data = target_data[target_data.receiver_aux_loss == False]

NUM_AGENTS = 1
target_data = target_data[target_data.num_agents == NUM_AGENTS]

MAX_LEN = 10
target_data = target_data[target_data.max_len == MAX_LEN]

VOCAB_SIZE = 2
target_data = target_data[target_data.vocab_size == VOCAB_SIZE]

VOCAB_SIZE_FEEDBACK = 2
target_data = target_data[(target_data.vocab_size_feedback == VOCAB_SIZE_FEEDBACK) | (target_data.feedback == False)]

LAYER_NORM = 1
target_data = target_data[target_data.layer_norm == LAYER_NORM]

DISCRIMINATION_NUM_OBJECTS = 2
target_data = target_data[target_data.discrimination_num_objects == DISCRIMINATION_NUM_OBJECTS]

ATTR_VAL = "4_4"
# ATTR_VAL = "3_10"

target_data = target_data[(target_data["attr_val"] == ATTR_VAL)]

# print(target_data.dir_name.unique())
target_hparam = "noise"
order = [0, 0.1, 0.3, 0.5, 0.7, 0.9]


num_runs_data = target_data.groupby(["attr_val", "condition"]).size().reset_index()

metrics = ["test_acc", "test_acc_no_noise", "topsim"]
# metrics = ["test_acc", "topsim", "topsim_sender_receiver", "posdis", "bosdis", "test_acc_no_noise", "train_acc_no_noise"]
_, axes = plt.subplots(1, len(metrics), figsize=(35, 12))

hue_variable = "feedback"
for value in order:
    for fb in [False, True]:
        d = target_data[(target_data[target_hparam] == value) & (target_data[hue_variable] == fb)]
        if len(d) != 3:
            print(f"Unexpected number of datapoints: {len(d)}")
            print(value)
            print("fb: ", fb)

for i, metric in enumerate(metrics):
    axis = axes[i]
    # g = sns.barplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue=hue_variable, dodge=True)
    g = sns.pointplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue=hue_variable, errorbar=('ci', 95))

    if not i == 0:
        g.legend().remove()
    else:
        sns.move_legend(axis, "lower left")
    axis.set_title(metric)
    axis.set_ylabel("")

plt.tight_layout()
plt.savefig("plots/effect_of_noise.pdf", dpi=300)

In [ ]:
target_data = data.copy()

target_data = target_data[target_data.receiver_aux_loss == False]

NUM_AGENTS = 1
target_data = target_data[target_data.num_agents == NUM_AGENTS]

NOISE = 0.7
target_data = target_data[target_data.noise == NOISE]

VOCAB_SIZE = 2
target_data = target_data[target_data.vocab_size == VOCAB_SIZE]

VOCAB_SIZE_FEEDBACK = 2
target_data = target_data[(target_data.vocab_size_feedback == VOCAB_SIZE_FEEDBACK) | (target_data.feedback == False)]

LAYER_NORM = 1
target_data = target_data[target_data.layer_norm == LAYER_NORM]

DISCRIMINATION_NUM_OBJECTS = 2
target_data = target_data[target_data.discrimination_num_objects == DISCRIMINATION_NUM_OBJECTS]

ATTR_VAL = "4_4"

target_data = target_data[(target_data["attr_val"] == ATTR_VAL)]

# target_hparam = "attr_val"
# order = ["2_10", "4_4", "2_100", "2_1000"]
target_hparam = "max_len"
order = [1, 3, 5, 10, 20, 30, 50]


num_runs_data = target_data.groupby(["attr_val", "condition"]).size().reset_index()

metrics = ["test_acc", "test_acc_no_noise", "topsim"]
# metrics = ["test_acc", "topsim", "topsim_sender_receiver", "posdis", "bosdis", "test_acc_no_noise", "train_acc_no_noise"]
_, axes = plt.subplots(1, len(metrics), figsize=(35, 12))

hue_variable = "feedback"
for value in order:
    for fb in [False, True]:
        d = target_data[(target_data[target_hparam] == value) & (target_data[hue_variable] == fb)]
        if len(d) != 3:
            print(f"Unexpected number of datapoints: {len(d)}")
            print(value)
            print(fb)
            
for i, metric in enumerate(metrics):
    axis = axes[i]
    # g = sns.barplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue="feedback", dodge=True)
    g = sns.pointplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue=hue_variable, errorbar=('ci', 95))

    if not i == 0:
        g.legend().remove()
    else:
        sns.move_legend(axis, "upper left")
    axis.set_title(metric)
    axis.set_ylabel("")
    axis.set_xlabel("message_length")


plt.tight_layout()
plt.savefig("plots/effect_of_message_length.pdf", dpi=300)

In [ ]:
target_data = data.copy()

target_data = target_data[target_data.guesswhat == True]

target_data = target_data[target_data.receiver_output_attention == False]
target_data = target_data[target_data.receiver_object_attention == False]
target_data = target_data[target_data.sender_attention == False]

target_data = target_data[(target_data.receiver_aux_loss == False)]

NUM_AGENTS = 1
target_data = target_data[target_data.num_agents == NUM_AGENTS]

MAX_LEN = 10
target_data = target_data[target_data.max_len == MAX_LEN]

VOCAB_SIZE = 2
target_data = target_data[target_data.vocab_size == VOCAB_SIZE]

VOCAB_SIZE_FEEDBACK = 2
target_data = target_data[(target_data.vocab_size_feedback == VOCAB_SIZE_FEEDBACK) | (target_data.feedback == False)]

LAYER_NORM = 1
target_data = target_data[target_data.layer_norm == LAYER_NORM]

DISCRIMINATION_NUM_OBJECTS = 10
target_data = target_data[target_data.discrimination_num_objects == DISCRIMINATION_NUM_OBJECTS]

# target_hparam = "attr_val"
# order = ["2_10", "4_4", "2_100", "2_1000"]
target_hparam = "noise"
order = [0, 0.1, 0.3, 0.5, 0.7, 0.9]

metrics = ["test_acc"] #test_acc_no_noise
_, axes = plt.subplots(1, len(metrics)*2, figsize=(35, 12))

hue_variable = "feedback"
for value in order:
    for fb in [False, True]:
        d = target_data[(target_data[target_hparam] == value) & (target_data[hue_variable] == fb)]
        if len(d) != 3:
            print(f"Unexpected number of datapoints: {len(d)}")
            print(value)
            print(fb)
            
for i, metric in enumerate(metrics):
    axis = axes[i]
    # g = sns.barplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue="feedback", dodge=True)
    g = sns.pointplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue=hue_variable, errorbar=('ci', 95))

    if not i == 0:
        g.legend().remove()
    else:
        sns.move_legend(axis, "lower left")
    axis.set_title(metric)
    axis.set_ylabel("")
    # axis.set_ylim((0, 1))


###
# SECOND HALF
###

target_data = data.copy()

target_data = target_data[target_data.guesswhat == True]

target_data = target_data[target_data.receiver_output_attention == False]
target_data = target_data[target_data.receiver_object_attention == False]
target_data = target_data[target_data.sender_attention == False]

target_data = target_data[target_data.receiver_aux_loss == False]


NUM_AGENTS = 1
target_data = target_data[target_data.num_agents == NUM_AGENTS]

NOISE = 0.5
target_data = target_data[target_data.noise == NOISE]

VOCAB_SIZE = 2
target_data = target_data[target_data.vocab_size == VOCAB_SIZE]

VOCAB_SIZE_FEEDBACK = 2
target_data = target_data[(target_data.vocab_size_feedback == VOCAB_SIZE_FEEDBACK) | (target_data.feedback == False)]

LAYER_NORM = 1
target_data = target_data[target_data.layer_norm == LAYER_NORM]

DISCRIMINATION_NUM_OBJECTS = 10
target_data = target_data[target_data.discrimination_num_objects == DISCRIMINATION_NUM_OBJECTS]

# target_hparam = "attr_val"
# order = ["2_10", "4_4", "2_100", "2_1000"]
target_hparam = "max_len"
order = [1, 5, 10, 20, 30, 50]

metrics = ["test_acc"] #test_acc_no_noise

hue_variable = "feedback"
for value in order:
    for fb in [False, True]:
        d = target_data[(target_data[target_hparam] == value) & (target_data[hue_variable] == fb)]
        if len(d) != 3:
            print(f"Unexpected number of datapoints: {len(d)}")
            print(value)
            print(fb)

for i, metric in enumerate(metrics):
    axis = axes[i+len(metrics)]
    # g = sns.barplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue="feedback", dodge=True)
    g = sns.pointplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue=hue_variable, errorbar=('ci', 95))

    if not i == 0:
        g.legend().remove()
    else:
        sns.move_legend(axis, "upper left")
    axis.set_title(metric)
    axis.set_ylabel("")
    axis.set_xlabel("message_length")

# axis.set_ylim((0, 1))

plt.tight_layout()
plt.savefig("plots/guesswhat.pdf", dpi=300)

In [ ]:
target_data = data.copy()

target_data = target_data[target_data.guesswhat == True]

target_data = target_data[target_data.receiver_output_attention == False]
target_data = target_data[target_data.receiver_object_attention == False]
target_data = target_data[target_data.sender_attention == False]

target_data = target_data[(target_data.receiver_aux_loss == False)]

NUM_AGENTS = 1
target_data = target_data[target_data.num_agents == NUM_AGENTS]

MAX_LEN = 10
target_data = target_data[target_data.max_len == MAX_LEN]

VOCAB_SIZE = 2
target_data = target_data[target_data.vocab_size == VOCAB_SIZE]

VOCAB_SIZE_FEEDBACK = 2
target_data = target_data[(target_data.vocab_size_feedback == VOCAB_SIZE_FEEDBACK) | (target_data.feedback == False)]

LAYER_NORM = 1
target_data = target_data[target_data.layer_norm == LAYER_NORM]

DISCRIMINATION_NUM_OBJECTS = 10
target_data = target_data[target_data.discrimination_num_objects == DISCRIMINATION_NUM_OBJECTS]

# target_hparam = "attr_val"
# order = ["2_10", "4_4", "2_100", "2_1000"]
target_hparam = "noise"
order = [0, 0.1, 0.3, 0.5, 0.7, 0.9]

metrics = ["test_acc_no_noise"] #
_, axes = plt.subplots(1, len(metrics)*2, figsize=(35, 12))

hue_variable = "feedback"
for value in order:
    for fb in [False, True]:
        d = target_data[(target_data[target_hparam] == value) & (target_data[hue_variable] == fb)]
        if len(d) != 3:
            print(f"Unexpected number of datapoints: {len(d)}")
            print(value)
            print(fb)

for i, metric in enumerate(metrics):
    axis = axes[i]
    # g = sns.barplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue="feedback", dodge=True)
    g = sns.pointplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue=hue_variable, errorbar=('ci', 95))

    if not i == 0:
        g.legend().remove()
    else:
        sns.move_legend(axis, "lower left")
    axis.set_title(metric)
    axis.set_ylabel("")
    # axis.set_ylim((0, 1))


###
# SECOND HALF
###

target_data = data.copy()

target_data = target_data[target_data.guesswhat == True]

target_data = target_data[target_data.receiver_output_attention == False]
target_data = target_data[target_data.receiver_object_attention == False]
target_data = target_data[target_data.sender_attention == False]

target_data = target_data[target_data.receiver_aux_loss == False]


NUM_AGENTS = 1
target_data = target_data[target_data.num_agents == NUM_AGENTS]

NOISE = 0.5
target_data = target_data[target_data.noise == NOISE]

VOCAB_SIZE = 2
target_data = target_data[target_data.vocab_size == VOCAB_SIZE]

VOCAB_SIZE_FEEDBACK = 2
target_data = target_data[(target_data.vocab_size_feedback == VOCAB_SIZE_FEEDBACK) | (target_data.feedback == False)]

LAYER_NORM = 1
target_data = target_data[target_data.layer_norm == LAYER_NORM]

DISCRIMINATION_NUM_OBJECTS = 10
target_data = target_data[target_data.discrimination_num_objects == DISCRIMINATION_NUM_OBJECTS]

# target_hparam = "attr_val"
# order = ["2_10", "4_4", "2_100", "2_1000"]
target_hparam = "max_len"
order = [1, 5, 10, 20, 30, 50]

metrics = ["test_acc_no_noise"] #

hue_variable = "feedback"
for value in order:
    for fb in [False, True]:
        d = target_data[(target_data[target_hparam] == value) & (target_data[hue_variable] == fb)]
        if len(d) != 3:
            print(f"Unexpected number of datapoints: {len(d)}")
            print(value)
            print(fb)

for i, metric in enumerate(metrics):
    axis = axes[i+len(metrics)]
    # g = sns.barplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue="feedback", dodge=True)
    g = sns.pointplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue=hue_variable, errorbar=('ci', 95))

    if not i == 0:
        g.legend().remove()
    else:
        sns.move_legend(axis, "upper left")
    axis.set_title(metric)
    axis.set_ylabel("")
    axis.set_xlabel("message_length")

# axis.set_ylim((0, 1))

plt.tight_layout()
plt.savefig("plots/guesswhat_no_noise.pdf", dpi=300)

In [ ]:
target_data = data.copy()

target_data = target_data[target_data.guesswhat == True]

target_data = target_data[target_data.receiver_output_attention == False]
target_data = target_data[target_data.receiver_object_attention == False]
target_data = target_data[target_data.sender_attention == False]

target_data = target_data[target_data.receiver_aux_loss == False]


NUM_AGENTS = 1
target_data = target_data[target_data.num_agents == NUM_AGENTS]

NOISE = 0.5
target_data = target_data[target_data.noise == NOISE]

VOCAB_SIZE = 2
target_data = target_data[target_data.vocab_size == VOCAB_SIZE]

VOCAB_SIZE_FEEDBACK = 2
target_data = target_data[(target_data.vocab_size_feedback == VOCAB_SIZE_FEEDBACK) | (target_data.feedback == False)]

LAYER_NORM = 1
target_data = target_data[target_data.layer_norm == LAYER_NORM]

DISCRIMINATION_NUM_OBJECTS = 10
target_data = target_data[target_data.discrimination_num_objects == DISCRIMINATION_NUM_OBJECTS]

# target_hparam = "attr_val"
# order = ["2_10", "4_4", "2_100", "2_1000"]
target_hparam = "max_len"
order = [1, 5, 10, 20, 30, 50]

metrics = ["test_acc", "test_acc_no_noise", "train_acc"]
# metrics = ["test_acc", "topsim", "topsim_sender_receiver", "posdis", "bosdis", "test_acc_no_noise", "train_acc_no_noise"]
_, axes = plt.subplots(1, len(metrics), figsize=(35, 10))

hue_variable = "feedback"
for value in order:
    for fb in [False, True]:
        d = target_data[(target_data[target_hparam] == value) & (target_data[hue_variable] == fb)]
        if len(d) != 3:
            print(f"Unexpected number of datapoints: {len(d)}")
            print(value)
            print(fb)
            
for i, metric in enumerate(metrics):
    axis = axes[i]
    # g = sns.barplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue="feedback", dodge=True)
    g = sns.pointplot(ax=axis, data=target_data, x=target_hparam, y=metric, order=order, hue=hue_variable, errorbar=('ci', 95), dodge=0.1)

    if not i == 0:
        g.legend().remove()
    else:
        sns.move_legend(axis, "upper left")
    axis.set_title(metric)
    axis.set_ylabel("")
    axis.set_xlabel("message_length")

# axis.set_ylim((0, 1))

plt.tight_layout()
plt.savefig("plots/guesswhat_max_len.pdf", dpi=300)

In [ ]:
data = []

noise = [0.5324794832658659, 0.5335070968199509, 0.5335038136672091, 0.5339482142500649, 0.5332509060283926, 0.5333704351868614, 0.5327118284699583, 0.5300618360198712, 0.5303755071863462]
for i, n in enumerate(noise):
    data.append({"condition": "after noise token", "turn": i, "prob": n})
    
no_noise = [0.5354631303998393, 0.53442421325633, 0.53442849654834, 0.5339807338244494, 0.5346668254157241, 0.5345516733947143, 0.5352325882994804, 0.537851318115185, 0.5375381968005606]
for i, n in enumerate(no_noise):
    data.append({"condition": "after other token", "turn": i, "prob": n})



data = pd.DataFrame.from_records(data)

_, axes = plt.subplots(1, 2, figsize=(35, 12), sharey=True)
# plt.figure(figsize=(10, 10))
sns.pointplot(ax=axes[0], data=data, x="turn", y="prob", hue="condition")
# plt.ylim((0, 1))

data_2 = []
after_0 = [0.3780947411143967, 0.3950762783997905, 0.4079030503418036, 0.4179165733900455, 0.41563186322742424, 0.4177290918843091, 0.42253313184999913, 0.4264756773331581, 0.4233993503091271]
for i, n in enumerate(after_0):
    data_2.append({"condition": "after 0", "turn": i, "prob": n})

after_1 = [0.4266391045795913, 0.4277036238071697, 0.42208971246220744, 0.4135762664003001, 0.41550587560905705, 0.41043735842481266, 0.4081567511616348, 0.4106360314442512, 0.4131322223131404]
for i, n in enumerate(after_1):
    data_2.append({"condition": "after 1", "turn": i, "prob": n})

data_2 = pd.DataFrame.from_records(data_2)
sns.pointplot(ax=axes[1], data=data_2, x="turn", y="prob", hue="condition")

# plt.ylim((0, 1))
# sns.pointplot(x=turn, y=no_noise)
